In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.DCVC_net import DCVC_net
import torch
from torchvision import transforms
import numpy as np
import pathlib
import os
import matplotlib.pyplot as plt
import wandb
import tqdm

/home/jatin/miniconda3/envs/DCVC/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BATCH_SIZE = 1
DATA_DIR = pathlib.Path('/data2/jatin/vimeo_septuplet/sequences')
DEVICE = torch.device('cuda')
DEVICE

device(type='cuda')

In [4]:
video_net = DCVC_net(up_strategy='default')

In [5]:
# load the good weights
# video_net.opticFlow = torch.load('../DCVC/optflow.pth')
# video_net.mvEncoder = torch.load('../DCVC/mvenc.pth')
# video_net.mvDecoder_part1 = torch.load('../DCVC/mvDecoder_part1.pth')
# video_net.mvDecoder_part2 = torch.load('../DCVC/mvDecoder_part2.pth')
# video_net.feature_extract = torch.load('../DCVC/feature_extract.pth')
# video_net.context_refine = torch.load('../DCVC/context_refine.pth')

# chpt = torch.load('dcvc-b-frame-with-bitrate/dcvc_epoch=2_int_really_good.pt')
chpt = torch.load('dcvc-b-frame-with-bitrate/dcvc_epoch=2_int_allquantize.pt')
video_net.load_state_dict(chpt['model'], strict=True)

# temporalPriorEncoder = video_net.temporalPriorEncoder
# del video_net.temporalPriorEncoder
# chpt = torch.load('dcvc-b-frame-convtranspose-fail/dcvc_epoch=4_int.pt')
# video_net.load_state_dict(chpt['model'], strict=False)
# video_net.temporalPriorEncoder = temporalPriorEncoder

video_net = video_net.to(DEVICE)
# optimizer = torch.optim.Adam(video_net.parameters(), lr=wandb.config.learning_rate)
# optimizer.load_state_dict(chpt['optimizer'])

# video_net.opticFlow.requires_grad_ = False
# video_net.mvEncoder.requires_grad_ = False
# video_net.mvDecoder_part1.requires_grad_ = False
# video_net.mvDecoder_part2.requires_grad_ = False
# video_net.feature_extract.requires_grad_ = False
# video_net.context_refine.requires_grad_ = False

In [6]:
optimizer = torch.optim.Adam(video_net.parameters(), lr=1e-4)
# optimizer.load_state_dict(chpt['optimizer'])
del chpt

In [7]:
class VideoDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, crop_size=256, make_b_cut=True, deterministic=False):
        self.data_dir = data_dir
        self.crop_size = crop_size
        self.make_b_cut = make_b_cut
        self.deterministic = deterministic
        self.all_paths = []
        for seq in os.listdir(self.data_dir):
            subseq = os.listdir(self.data_dir / seq)
            for s in subseq:
                self.all_paths.append(self.data_dir / seq / s)
        assert len(self.all_paths) == 91701
        
        self.transforms = torch.nn.Sequential(
            transforms.RandomCrop(crop_size)
        )
       
    def __getitem__(self, i):
        path = self.all_paths[i]
        imgs = []
        if self.make_b_cut:
            # load two reference frames and the B-frame in the middle
            #TODO: implement making this deterministic
            interval = np.random.randint(1, 4) # can be 1, 2, or 3
            ref1 = plt.imread(path / f'im{1}.png')
            ref2 = plt.imread(path / f'im{1 + interval*2}.png')
            # this is the B-frame, in the middle
            im = plt.imread(path / f'im{1 + interval}.png')
            imgs = [ref1, ref2, im]
        else:
            # load full sequence
            for i in range(1, 8):
                # should be between [0, 1]
                img = plt.imread(path / f'im{i}.png')
        
        # plt.imread should make inputs in [0, 1] for us
        imgs = np.stack(imgs, axis=0)
        # bring RGB channels in front
        imgs = imgs.transpose(0, 3, 1, 2)
        return self.transforms(torch.FloatTensor(imgs))

    def __len__(self):
        return len(self.all_paths)

ds = VideoDataset(DATA_DIR)
dl = torch.utils.data.DataLoader(
    ds,
    shuffle=True,
    batch_size=BATCH_SIZE,
    num_workers=6,
    prefetch_factor=5
)

In [8]:
def count_parameters(model):
    """Return number of parameters in a model"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(video_net)} trainable parameters')

The model has 10622976 trainable parameters


In [9]:
dl_iter = iter(dl)

In [10]:
x = next(dl_iter)

In [11]:
x = x.to(DEVICE)
ref1 = x[:,0]
ref2 = x[:,1]
im = x[:,2]

In [12]:
preds_dict = video_net(ref1, ref2, im, compress_type='train_compress')
loss = preds_dict['bpp']
print(loss.item())
loss.backward()

3.3197596073150635


In [14]:
for _ in range(100):
    preds_dict = video_net(ref1, ref2, im, compress_type='train_compress')
    loss = preds_dict['bpp']
    print(loss.item())
    loss.backward()
    optimizer.step()

2.9754481315612793
2.7353761196136475
2.521721839904785
2.341061592102051
2.1811470985412598
2.0431535243988037
1.9324393272399902
1.8288992643356323
1.7448865175247192
1.6608995199203491
1.592614769935608
1.5293642282485962
1.463364839553833
1.4528388977050781
1.4479596614837646
1.444175124168396
1.4095598459243774
1.3490550518035889
1.286040186882019
1.2346278429031372
1.212756633758545
1.2182588577270508
1.2370195388793945
1.238594651222229
1.2075483798980713
1.1560213565826416
1.1070743799209595
1.086803913116455
1.0694774389266968
1.0522565841674805
1.04745352268219
1.044086217880249
1.0309712886810303
1.008552074432373
1.011487603187561
1.007869839668274
1.0046440362930298
1.0100911855697632
1.0162183046340942
1.0141466856002808
1.0208611488342285
1.019641399383545
1.0139492750167847
1.0039151906967163
0.9946315884590149
1.0241719484329224
1.046319603919983
1.0741026401519775
1.0890889167785645
1.0669846534729004
1.0319241285324097
0.9645816087722778


KeyboardInterrupt: 